# Boilerplate

In [17]:
import pandas as pd 
import pandasql as ps
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import scipy.stats as stats
import statsmodels.stats.api as sms

from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

def f():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    
def nf():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 4)

import warnings
warnings.filterwarnings('ignore')

def dp(df, r = 5, c = None):
    from IPython.display import display
    with pd.option_context('display.max_rows', 4, 'display.max_columns', None):
        display(df)

def fg(w = 6, h = 4, dpi = 120):
    plt.rcParams['figure.figsize'] = (w,h)
    plt.rcParams['figure.dpi'] = dpi
fg()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create sample

In [18]:
import random 
import string 

# random sample selection

In [20]:
# letters = string.ascii_uppercasedfall = df
letters = string.ascii_uppercase[:5]

df = pd.DataFrame()
_len = 2000
for i in range(5):
    df[f'Q{i}'] = [''.join(random.choice(letters) for _ in range(1)) for _ in range(_len)]

# init
n_clu = 7
_cho = random.sample(df.index.to_list(),n_clu)

dfcho = df[df.index.isin(_cho)].reset_index(drop = True)

dforig = df.copy()
df = dforig.copy()

# To store every output of clusters
dfall = pd.DataFrame()
 
# compute difference thorught loopings
difflist = []

# to receive buckets of similarities
bucketlist = []

# to iterate over the bucketlist
cont = 0
_maxiter = 50
for _ in range(_maxiter):
    # restart df
    df = dforig.copy()
    # computacao ineficient O(n²) ou mais
#     print('cluster dataframe')
#     dfcho
    for idx in df.index:
        for clu in dfcho.index:
            df.loc[idx,f'C{clu}'] = sum(dforig.iloc[idx,:] == dfcho.iloc[clu,:])
            
    # Minimum distances to clusters
    dfclosest = df.select_dtypes(include = [float]).idxmin(axis = 1)

    closest = 'closest'
    df[closest] = dfclosest

    clusters = df.select_dtypes(include = [float]).columns

    # for clu in clusters:
    #     df[df.closest == clu]

    # Return mode

    _dforig = dforig.copy()
    _dforig[closest] = df[closest]

    dfmode = pd.DataFrame()
    for clu in clusters:
        dfmode = dfmode.append(_dforig[_dforig.closest == clu].mode(axis = 0).iloc[0,:])
#         print('loop dfmode')
#         dfmode

    # new clusters
    dfmode = dfmode.drop(closest, axis = 1)
    
#     print('Clusters found')
#     dfmode
    
    del dfcho
    dfcho = dfmode.copy().reset_index(drop = True)
    
    dfall = dfall.append(dfcho)
    
    # Compute similarities
    dfclosest = dfclosest.reset_index().rename(columns = {0:'cluster'})
    buckets = {}
    for clu in dfcho.index:
        buckets[f'C{clu}'] = dfclosest[dfclosest.cluster == f'C{clu}'].index.to_list()
    bucketlist.append(buckets)
    
    # only computes distance from secon iteration
    if cont >=1:
        diff = 0
        for clu in dfcho.index:
            _clu = f'C{clu}'
            diff += len(set(bucketlist[cont].get(_clu))^set(bucketlist[cont-1].get(_clu)))
#             print(f'difference{diff}')
        difflist.append(diff)
        print(f'current{difflist[cont-1]} and former {difflist[cont-2]}')
    
    try:
        if (cont > 2) & (difflist[cont-1] == difflist[cont-2]):
            print('Convergence reached')
            break
    except IndexError:
        print('out of range')

    cont +=1

out of range
current3634 and former 3634
current3682 and former 3634
current3690 and former 3682
current3712 and former 3690
current3722 and former 3712
current3722 and former 3722
Convergence reached


In [21]:
difflist

[3634, 3682, 3690, 3712, 3722, 3722]